# 1. 卷积神经网络及反向传播

## 卷积神经网络
### 卷积层
![convolution](./images/convolution.png)

### 池化层
![max pooling](./images/pooling.png)

### 全连接层
![fully connected layer](./images/fully_connect.jpg)


## 矩阵&向量偏导计算
1. 向量对向量的偏导计算

$$y=Wx \quad \frac{d\vec{y}}{d\vec{x}}=W$$

$$y=xW \quad \frac{d\vec{y}}{d\vec{x}}=W^{T}$$

2. 向量对矩阵的偏导计算

$$y=Wx \quad \frac{d\vec{y_{i}}}{dW_{i,j}}=d\vec{x_{i}}$$


3. 矩阵对矩阵的偏导计算

$$Y=WX \quad \frac{d\vec{Y_{i,:}}}{d\vec{X_{i,:}}}=W^{T}$$


## 全连接层
$\sigma$-激活函数, $C$-损失函数, $a^{l}$-第l层激活值, $z^{l}$-第l层加权和, $\mathbf{W}^{l}$-第l层权重, $b^{l}$-第l层偏置
### 前向传播
$$ z^{l}=\mathbf{W}^{l}a^{l-1}+b^{l} $$
$$ a^{l}=\sigma(z^{l}) $$
$$ C=\frac{1}{2}||a^{L}-y||^{2} $$

### 反向传播
输出层L
$$ \delta^{L}=\frac{\partial C}{\partial a^{L}}\frac{\partial a^{L}}{\partial z^{L}}=(a^{L}-y) \cdot \sigma^{'}(z^{L})$$
$$ \frac{\partial C}{\partial W^{L}}=\frac{\partial C}{\partial z^{L}}\frac{\partial z^{L}}{\partial W^{L}}=\delta^{L}(a^{L-1})^{T} $$
$$ \frac{\partial C}{\partial b^{L}}=\frac{\partial C}{\partial z^{L}}\frac{\partial z^{L}}{\partial b^{L}}=\delta^{L} $$
隐藏层
$$ \delta^{l}=\frac{\partial C}{\partial z^{l}}=\frac{\partial C}{\partial z^{l+1}}\frac{\partial z^{l+1}}{\partial z^{l}}=\delta^{l+1}\frac{\partial z^{l+1}}{\partial z^{l}}=(W^{l+1})^{T}\delta^{l+1} \cdot \sigma^{'}(z^{l}) $$
$$ \frac{\partial C}{\partial W^{l}}=\frac{\partial C}{\partial z^{l}}\frac{\partial z^{l}}{\partial W^{l}}=\delta^{l}(a^{l-1})^{T} $$
$$ \frac{\partial C}{\partial b^{l}}=\frac{\partial C}{\partial z^{l}}\frac{\partial z^{l}}{\partial b^{l}}=\delta^{l} $$

### 参数更新
$\eta$-学习率，取值范围(0,1)
$$ W^{l}=W^{l}-\eta\frac{\partial C}{\partial W^{l}} $$
$$ b^{l}=b^{l}-\eta\frac{\partial C}{\partial b^{l}} $$
batch更新
$$ W^{l}=W^{l}-\eta\frac{1}{m}\sum_{i=1}^{m}\frac{\partial C}{\partial W^{l}} $$
$$ b^{l}=b^{l}-\eta\frac{1}{m}\sum_{i=1}^{m}\frac{\partial C}{\partial b^{l}} $$
momentum更新，$\gamma$-动量系数，取值范围(0,1)，代表指数衰减平均
$$ v_{t}=\gamma v_{t-1}+\eta\frac{\partial C}{\partial W^{l}} $$
$$ W^{l}=W^{l}-v_{t} $$


### 其他
#### 激活函数的作用
拟合非线性变换，如果没有（非线性）激活函数，那么多层神经网络就是一个线性变换，无法拟合复杂的非线性函数

#### 正则化 - 防止过拟合
$$ C=C_{0}+\frac{\lambda}{2}||W||^{2} $$
$\lambda$ - 正则化系数

#### 梯度爆炸与梯度消失

1. 两种情况下梯度消失经常出现，一是在深层网络中，二是采用了不合适的损失函数，比如sigmoid。梯度爆炸一般出现在深层网络和权值初始化值太大的情况下。

- 深层网络角度
    - 对激活函数求导，如果大于1，层数越多，梯度越大，梯度爆炸；如果小于1，层数越多，梯度越小，梯度消失
    - 不同的层学习的速度差异很大，表现为网络中靠近输出的层学习的情况很好，靠近输入的层学习的很慢
- 激活函数角度
    - sigmoid、tanh的导数永远小于1，层数越多，梯度越小，梯度消失

2. 解决方案
- 使用梯度裁剪、正则：解决梯度爆炸
- 使用ReLU、LeakyReLU、ELU等激活函数
- 使用Batch Normalization：反向传播式子中有$W$的存在，会影响梯度的消失和爆炸，batchnorm通过对每一层的输出做scale和shift的方法，把每层神经网络的输入值拉回到均值为0方差为1的分布，使得激活输入值落在非线性函数对输入比较敏感的区域，避免梯度消失
- 使用残差网络
$$ y=F(z)+z $$
$$ \frac{\partial loss}{\partial z_{l}} = \frac{\partial loss}{\partial z_{L}}\frac{\partial z_{L}}{\partial z_{l}} = \frac{\partial loss}{\partial z_{L}} (1+\frac{\partial }{\partial z_{l}}\sum_{i=l}^{L-1}F(z_{i})) $$

## Pytorch实现

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import random

# 

3
